In [ ]:
# xmlrpc_server.ipynb

import matplotlib.pyplot as plt
from pylab import *


from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc
import datetime
import pandas as pd
import pickle
import os
import requests


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

server.register_introspection_functions()

stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")
def add_log_db(log_line, time):
    worktime = datetime.datetime.now() - time
    worktime = worktime.microseconds
    #worktime = worktime.seconds().strftime("%S")
    stats_server.add_log_db(log_line, time, worktime)
    
    try:
        stats_server.add_log_db(log_line, time, worktime)
        
    except:
        print('Server is off')
    return True
server.register_function(add_log_db, 'add_log_db')


def add_log(log_line):
    check = 0
    try:
        r = requests.get('http://127.0.0.1:8018')
        check = r.status_code
    except:
        print("Stats server is not connected")
    if(check == 501):
        stats_server.add_log(log_line)
    return True
# Тест
def ping():
    time = datetime.datetime.now()
    add_log_db('ping', time)
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    time = datetime.datetime.now()
    add_log_db('now', time)
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    time = datetime.datetime.now()
    add_log_db('type', time)
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    time = datetime.datetime.now()
    add_log_db('sum', time)
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log_db('pow', datetime.datetime.now())
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    time = datetime.datetime.now()
    #add_log('name_checker')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    boy = "good boy"
    if (exist == True):
        boy = "bad_boy"
    add_log_db('name_checker', time)
    return sname + ": "+ boy

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    time = datetime.datetime.now()
    data = bin_data.data
    add_log_db('binar_send', time)
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    time = datetime.datetime.now()
    
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    colors = len(img_arr.shape)
    
    if (colors == 3):
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
    elif(colors == 2):
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    pimg = pickle.dumps(img_arr)
    add_log_db('inversion', time)
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

def image_mirrorring(bin_data):
    time = datetime.datetime.now()
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1] - 1
    
    for i in range(height):
        for j in range(width // 2):
            original = img_arr[i][j].copy()
            img_arr[i][j] = img_arr[i][width - j].copy()
            img_arr[i][width - j] = original
    pimg = pickle.dumps(img_arr)
    add_log_db('mirroring', time)
    
    return xmlrpc.client.Binary(pimg)
server.register_function(image_mirrorring, 'image_mirrorring')
 
def binarisation_image(bin_data, porog):
    time = datetime.datetime.now()
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    colors = len(img_arr.shape)

    for i in range(height):
        for j in range(width):
            if(colors == 3):
                if(img_arr[i][j][0] > porog and img_arr[i][j][1] > porog and img_arr[i][j][2] > porog):
                    img_arr[i][j][0] = 255
                    img_arr[i][j][1] = 255
                    img_arr[i][j][2] = 255
                else:
                    img_arr[i][j][0] = 0
                    img_arr[i][j][1] = 0
                    img_arr[i][j][2] = 0
            elif(colors == 2):
                if(img_arr[i][j] > porog):
                    img_arr[i][j] = 255
                else:
                    img_arr[i][j] = 0
    pimg = pickle.dumps(img_arr)
    add_log_db('binarization', time)
    return xmlrpc.client.Binary(pimg)
server.register_function(binarisation_image, 'binarisation_image')
    
def fio_birth_check(surname, name, parentname, birth):
    time = datetime.datetime.now()
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any((frame['Name'] == name) & (frame['Surname'] == surname) & (frame['Patronym'] == parentname) & (frame['Birth'] == birth))
    boya = "good boy"
    if (exist==True):
        boya = "bad boy"
    add_log_db('fiobirthcheck', time)
    
    return surname + " - " + boya
server.register_function(fio_birth_check, 'fio_birth_check')    

def get_journal_from_server(sname):
    return stats_server.get_journal(sname)
    
server.register_function(get_journal_from_server, 'get_journal_from_server')

def get_journal_from_server_db(sname):
    return stats_server.get_journal_db(sname)
server.register_function(get_journal_from_server_db, 'get_journal_from_server_db')

print ("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [22/Dec/2022 16:31:24] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2022 16:31:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2022 16:31:52] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2022 16:32:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2022 16:32:20] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2022 16:34:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2022 16:34:32] "POST /RPC2 HTTP/1.1" 200 -
